In [26]:
%matplotlib inline
import os,random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as cPickle, random, sys, keras
import scipy
from sklearn import preprocessing
import tensorflow as tf
from numpy.fft import *


from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

Xd = cPickle.load(open("RML2016.10b.dat",'rb'), encoding='latin1')

In [29]:
INTENSITY_AXIS=0
QUADRATURE_AXIS=1

snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])

X_Intensity = []  
X_Quadrature = []
Label_strings = []
for mod in mods:
    X_Intensity.append(Xd[(mod,18)][:,INTENSITY_AXIS,:])
    X_Quadrature.append(Xd[(mod,18)][:,QUADRATURE_AXIS,:])
    for label in range(len(Xd[(mod,18)][:,INTENSITY_AXIS,:])):
        Label_strings.append(mod)
X_Intensity = np.vstack(X_Intensity) # Quick way to create stacked dataset
X_Quadrature = np.vstack(X_Quadrature)

Label_strings=np.vstack(Label_strings)
complex_num = X_Intensity + 1j*X_Quadrature


In [30]:
X_Intensity_FFT = abs(np.fft.fftshift(fft(X_Intensity[:]-np.mean(X_Intensity[:]))))
X_Quadrature_FFT = abs(np.fft.fftshift(fft(X_Quadrature[:]-np.mean(X_Quadrature[:]))))
X_complex_FFT = abs(np.fft.fftshift(fft(complex_num[:]-np.mean(complex_num[:]))))

X_Power_Intensity_FFT = X_Intensity_FFT**2
X_Power_Quadrature_FFT = X_Quadrature_FFT**2
X_Power_complex_FFT = X_complex_FFT**2

le= preprocessing.LabelEncoder()
le.fit(Label_strings)
Label_Y_numbers = le.transform(Label_strings)
#depth = 10
#Label = tf.one_hot(Label_Y_numbers,depth)

In [31]:
Label_Y_numbers

array([0, 0, 0, ..., 9, 9, 9])

In [32]:
X_Intensity_Label = np.c_[X_Intensity_FFT, Label_Y_numbers]
X_Quadrature_Label = np.c_[X_Quadrature_FFT,Label_Y_numbers]
X_complex_Label = np.c_[X_complex_FFT,Label_Y_numbers]

X_Intensity_Label.shape

(60000, 129)

In [33]:
import copy
X_Intensity_Label_shuffled = copy.copy(X_Intensity_Label) # Create another copy to shuffle
X_Quadratue_Label_shuffled = copy.copy(X_Quadrature_Label)
X_complex_Label_shuffled = copy.copy(X_complex_Label)

np.random.shuffle(X_Intensity_Label_shuffled)
np.random.shuffle(X_Quadratue_Label_shuffled)
np.random.shuffle(X_complex_Label_shuffled)

length_of_X_train = int(0.7*X_Intensity_Label_shuffled.shape[0]) #42000
length_of_X_validation = (int(0.2*X_Intensity_Label_shuffled.shape[0]))+length_of_X_train #12000
length_of_X_train

42000

# Support Vector Machine 

In [38]:
Train_Intensity = X_Intensity_Label_shuffled[0:length_of_X_train,0:128]
Y_Intensity = X_Intensity_Label_shuffled[0:length_of_X_train,128:]

Validation_Intensity = X_Intensity_Label_shuffled[length_of_X_train:length_of_X_validation,0:128]
Y_Validation = X_Intensity_Label_shuffled[length_of_X_train:length_of_X_validation,128:]


In [36]:
Y_Intensity_1D = np.ravel(Y_Intensity,order = 'C')
Y_Intensity_1D

array([1., 4., 1., ..., 6., 8., 2.])

In [39]:
from sklearn import svm
# Creating a SVM Classifier 
svm_classifier_rbf = svm.SVC(kernel='rbf')
svm_classifier_rbf.fit(Train_Intensity,Y_Intensity_1D)

SVC()

In [40]:
y_pred_rbf=svm_classifier_rbf.predict(Validation_Intensity)#using Radial Base Function Kernal
y_pred_rbf

array([3., 8., 3., ..., 5., 6., 8.])

In [41]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(Y_Validation, y_pred_rbf))

Accuracy: 0.52275


In [42]:
Train_complex = X_complex_Label_shuffled[0:length_of_X_train,0:128]
Y_complex = X_complex_Label_shuffled[0:length_of_X_train,128:]

print(Train_complex.shape)
Validation_complex = X_complex_Label_shuffled[length_of_X_train:length_of_X_validation,0:128]
Y_validation_complex = X_complex_Label_shuffled[length_of_X_train:length_of_X_validation,128:]

(42000, 128)


In [43]:
Y_complex_1D = np.ravel(Y_complex,order='C')

In [44]:
from sklearn import svm
# Creating a SVM Classifier 
svm_classifier_rbf_Complex = svm.SVC(kernel='rbf')
svm_classifier_rbf_Complex.fit(Train_complex,Y_complex_1D)

SVC()

In [47]:
from sklearn import metrics
y_pred_rbf_complex=svm_classifier_rbf_Complex.predict(Validation_complex)

In [49]:
Y_validation_complex_1D = np.ravel(Y_validation_complex,order= 'C')
y_pred_rbf_complex

array([6., 2., 6., ..., 0., 3., 5.])

In [50]:
print("Accuracy:",metrics.accuracy_score(Y_validation_complex_1D, y_pred_rbf_complex))

Accuracy: 0.6760833333333334


# Random Forest Classifier

In [72]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 150)
classifier.fit(Train_complex, Y_complex_1D)

RandomForestClassifier(criterion='entropy', n_estimators=500, random_state=150)

In [73]:
y_pred = classifier.predict(Validation_complex)

In [74]:
print("Accuracy:",metrics.accuracy_score(Y_validation_complex_1D, y_pred))

Accuracy: 0.6859166666666666


In [ ]:
dual=[True,False]
max_iter=[100,110,120,130,140]
param_grid = dict(dual=dual,max_iter=max_iter)

lr = LogisticRegression(penalty='l2')
grid = GridSearchCV(estimator=lr, param_grid=param_grid, cv = 3, n_jobs=-1)
grid_result = grid.fit(X, y)